In [1]:
from molmap import loadmap

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
#IPythonConsole.ipython_useSVG = True
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

In [2]:
def sort_dict(x):
    return {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

def count_place(inh, parameter):
    count = {}
    for s in inh[parameter].unique():
        count[s] = sum(inh[parameter]==s)
    count = sort_dict(count)
    return count

from molmap import loadmap

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit import RDLogger
#IPythonConsole.ipython_useSVG = True
import numpy as np
import pandas as pd
from tqdm import tqdm
from openbabel import pybel
RDLogger.DisableLog('rdApp.*')
def clean_and_standardize(smiles,ph=7.4,iso=False):
    try:
        # Convert SMILES to RDKit molecule
        mol = Chem.MolFromSmiles(smiles)
        
        # Skip invalid molecules
        if mol is None:
            return None,None

        # Canonicalize the SMILES
        # canonical_smiles = Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)

        # Remove salts and other fragments / Keep only the largest fragment
        fragments = Chem.GetMolFrags(mol, asMols=True)
        largest_fragment = max(fragments, default=None, key=lambda m: m.GetNumAtoms())
        if largest_fragment is None:
            return None,None
        
        u = rdMolStandardize.Uncharger()
        uncharge_mol = u.uncharge(largest_fragment)
        uncharge_smiles = Chem.MolToSmiles(uncharge_mol, isomericSmiles=iso, canonical=True)
        
        ob_mol = pybel.readstring("smi", Chem.MolToSmiles(largest_fragment, isomericSmiles=iso, canonical=True))
        
        ob_mol.OBMol.AddHydrogens(False, True, ph)

        # Convert back to SMILES
        adjusted_smiles = ob_mol.write("smi").strip()

        return adjusted_smiles, uncharge_smiles
    
    except Exception as e:
        print(f"Error processing SMILES {smiles}: {e}")
        return None,None

In [3]:
df = pd.read_csv('./source/invivo.csv',encoding='utf-8')
# df = df[~(df['drug'].isna()) | ~(df['structure'].isna())]
# df = df[~df['value'].isna()]

In [4]:
df.columns

Index(['_id', 'drug', 'structure', 'transporter', 'species', 'geneName',
       'geneId', 'type', 'place', 'method', 'experimentalSystem', 'dose',
       'route', 'substanceMeasured', 'concomitant', 'time', 'comment',
       'parameterType', 'parameter', 'value', 'units', 'reference', 'source',
       'sourceLink', 'year', 'TransporterInfo_id', 'fullname',
       'structurelink'],
      dtype='object')

In [5]:
len(df)

201

In [6]:
df = df.dropna(subset=['method','place','geneName','type','structure','value','parameter','substanceMeasured','experimentalSystem'])

In [7]:
refine = ['drug','place','experimentalSystem','substanceMeasured','concomitant','value','reference','sourceLink']

In [8]:
# inh = df[df['type'] == 'Inhibitor'][refine]
# sub = df[df['type'] == 'Substrate'][refine]
inh = df[df['type'] == 'Inhibitor']
inh=inh[inh['parameter']!='AUC change']
sub = df[df['type'] == 'Substrate']

In [9]:
count_place(inh,'method')

{'In Vivo': 200}

In [10]:
count_place(inh,'geneName')

{'ABCB1': 200}

In [11]:
inh['clean_smiles_pH'] = [ clean_and_standardize(s)[1] for s in inh['structure'] ]
len(inh),len(inh.drop_duplicates(subset=['clean_smiles_pH']))

(200, 67)

In [12]:
count_place(inh,'parameter')

{'AUC increase': 95, 'AUC ratio': 82, 'AUC decrease': 23}

In [13]:
import re
def string2value(inh_cell_fda):
    l=[]
    print('origin:',len(inh_cell_fda))
    for s in inh_cell_fda['value']:
        l.append(re.findall(r"(?:\d*\.*\d+)", s))

    for i in range(len(l)):
        t = l[i]
        for j in range(len(t)):
            t[j] = float(t[j])

        if (len(t)==0):
            l[i]=None
            continue
            
        if (len(t)==1):
            total = float(t[0])
        elif (len(t)==2):
            total = sum([float(s) for s in t])/2
            
        if total < 1e-5 or total > 1000:
            total=None

        l[i]=total
    
    l = np.array(l)
    inh_cell_fda['value_num']=l
            
    inh_cell_fda=inh_cell_fda.dropna(subset=['value_num'])
    
    print('process:',len(inh_cell_fda))
    return inh_cell_fda

In [14]:
inh=string2value(inh)

origin: 200
process: 200


In [15]:
count_place(inh,'units')

{'%': 136, 'fold': 63, 'dimensionless': 1}

In [16]:
def reset_df(inh):
    inh_cell_fda=string2value(inh)
    inh_cell_fda = inh_cell_fda[inh_cell_fda['geneName']!='abcb4']
    len(inh_cell_fda),len(inh_cell_fda.drop_duplicates(subset=['clean_smiles_pH']))
    
    data = []
    value = []
    for (s,v) in zip(inh_cell_fda['units'],inh_cell_fda['value_num']):
        if ('%' in s):
            value.append(v/100)
        else:
            value.append(v)
        data.append('fold')
    inh_cell_fda['units'] = data
    inh_cell_fda['value_num'] = value

    data = []
    unit = []
    value = []
    for (s,v) in zip(inh_cell_fda['parameter'],inh_cell_fda['value_num']):
        if ('ratio' in s):
            value.append(v)
        elif 'increase' in s:
            value.append(1.0+v)
        else:
            value.append(1.0-v)
        data.append('ratio')
    inh_cell_fda['parameter'] = data
    inh_cell_fda['value_num'] = value

    print(len(inh_cell_fda),count_place(inh_cell_fda,'parameter'))
        
    return inh_cell_fda,None

In [17]:
from scipy import stats

In [18]:
inh_cell_fda,_=reset_df(inh)
data=defaultdict(list)
smi_dict=defaultdict(str)
for i in range(len(inh_cell_fda)):
    exp = inh_cell_fda.iloc[i]['drug']
    smi = inh_cell_fda.iloc[i]['clean_smiles_pH']
    sub = inh_cell_fda.iloc[i]['substanceMeasured']
    # val_o = inh_cell_fda.iloc[i]['value']
    val = inh_cell_fda.iloc[i]['value_num']

    data[smi+'_'+sub].append(val)
    smi_dict[smi+'_'+sub]=exp
print()

for k,v in data.items():
    print(smi_dict[k],round(np.array(v).mean(),3),'+-',round(np.array(v).std(),3))

origin: 200
process: 200
200 {'ratio': 200}

Ramelteon 0.97 +- 0.0
Eslicarbazepine Acetate 0.96 +- 0.0
Diroximel Fumarate 0.9 +- 0.0
Bosentan 0.925 +- 0.078
Prasugrel Hydrochloride 0.87 +- 0.0
Paroxetine Mesylate 0.85 +- 0.0
Tegaserod Maleate 0.85 +- 0.0
Obeticholic Acid 1.01 +- 0.0
Empagliflozin 1.06 +- 0.0
Saxagliptin Hydrochloride 1.06 +- 0.0
Edoxaban Tosylate 1.07 +- 0.0
Cobicistat 1.077 +- 0.0
Pexidartinib Hydrochloride 1.09 +- 0.0
Venetoclax 1.09 +- 0.0
Pantoprazole Sodium 1.104 +- 0.0
Gatifloxacin 1.306 +- 0.256
Sitagliptin Phosphate 1.11 +- 0.0
Zanubrutinib 1.113 +- 0.0
Lurasidone Hydrochloride 1.253 +- 0.174
Carvedilol 1.157 +- 0.024
Atorvastatin Calcium 1.15 +- 0.0
Nefazodone Hydrochloride 1.15 +- 0.0
Entrectinib 1.18 +- 0.0
Rabeprazole Sodium 1.19 +- 0.0
Canagliflozin 1.199 +- 0.002
Tolvaptan 1.208 +- 0.039
Istradefylline 1.21 +- 0.0
Flibanserin 1.762 +- 0.292
Isavuconazonium Sulfate 1.25 +- 0.0
Mirabegron 1.27 +- 0.0
Suvorexant 1.27 +- 0.0
Mibefradil 1.3 +- 0.0
Rolapitant H

In [19]:
inh_cell_fda,_=reset_df(inh)
print()
for keys in inh_cell_fda['substanceMeasured'].unique():
    data=defaultdict(list)
    flag=[]
    smi_dict=defaultdict(str)
    print(keys)
    for i in range(len(inh_cell_fda)):
        exp = inh_cell_fda.iloc[i]['drug']
        smi = inh_cell_fda.iloc[i]['clean_smiles_pH']
        sub = inh_cell_fda.iloc[i]['substanceMeasured']
        val_o = inh_cell_fda.iloc[i]['value']
        val = inh_cell_fda.iloc[i]['value_num']

        if ((keys == sub) and (
            'cein' in exp or
            'goxin' in exp or
            'damine' in exp or
            'blastine' in exp or
            'chst' in exp or
            'phine' in exp or
            'icine' in exp or
            'chst' in exp or
            'taxel' in exp or
            'bicin' in exp ) 
        ):
            # print(exp,val)
            flag.append(True)
            data[smi+'_'+sub].append(val)
            smi_dict[smi+'_'+sub]=exp
            # data.append(val)
        else:
            flag.append(False)
    
    test_list=[]
    for k,v in data.items():
        # if len(v)>1:
        #     test_list.append(v)
        p_value_max=round(stats.ttest_1samp(v, np.array(v).max())[1],3)
        p_value_min=round(stats.ttest_1samp(v, np.array(v).min())[1],3)
        print(smi_dict[k],k.split('_')[1],len(v),v,(round(np.array(v).mean(),3),round(np.array(v).std(),3)))
        # print((np.array(v).min(),np.array(v).max(),p_value_min,p_value_max))
    # if len(test_list)>1:
    #     print('mean',round(np.array([ s for t in test_list for s in t ]).mean(),3),'p_value',stats.f_oneway(*test_list)[1])
    print()

origin: 200
process: 200
200 {'ratio': 200}

Digoxin



In [20]:
inh_cell_fda,_=reset_df(inh)
data=defaultdict(list)
flag=[]
smi_dict=defaultdict(str)
for i in range(len(inh_cell_fda)):
    exp = inh_cell_fda.iloc[i]['drug']
    smi = inh_cell_fda.iloc[i]['clean_smiles_pH']
    sub = inh_cell_fda.iloc[i]['substanceMeasured']
    val_o = inh_cell_fda.iloc[i]['value']
    
    val = inh_cell_fda.iloc[i]['value_num']
    if (
        (
        ('Vinblast' in exp and
        'No' not in exp))
    ):
        flag.append(True)
        if val>2:
            data[smi+'_'+sub].append(val)
        smi_dict[smi+'_'+sub]=exp
    else:
        flag.append(False)
for k,v in data.items():
    print(smi_dict[k],k.split('_')[1],v,(round(np.array(v).mean(),3),round(np.array(v).std(),3)))

origin: 200
process: 200
200 {'ratio': 200}


In [21]:
inh_cell_fda,_=reset_df(inh)
data=defaultdict(list)
flag=[]
smi_dict=defaultdict(str)
for i in range(len(inh_cell_fda)):
    exp = inh_cell_fda.iloc[i]['drug']
    smi = inh_cell_fda.iloc[i]['clean_smiles_pH']
    sub = inh_cell_fda.iloc[i]['substanceMeasured']
    val = inh_cell_fda.iloc[i]['value_num']
    
    if (
        'qudar' in exp or
        'idar' in exp
    ):
        # print(exp,val)
        flag.append(True)
        data[smi+'_'+sub].append(val)
        smi_dict[smi+'_'+sub]=exp
        # data.append(val)
    else:
        flag.append(False)
for k,v in data.items():
    print(smi_dict[k],k.split('_')[1],v,min(v),max(v))

origin: 200
process: 200
200 {'ratio': 200}
Pexidartinib Hydrochloride Digoxin [1.09, 1.09] 1.09 1.09


In [22]:
def test_v1(data_dict,dropna=True,strict=True):
    from scipy.stats import chisquare,multinomial
    val_list = []
    for smi in data_dict.keys():
        
        temp = np.array(data_dict[smi])
        total=len(temp)
        
        results=np.zeros(3)
        for l in range(3):
            results[l]=sum(temp==l)
        
        observed=np.zeros(2)
        observed[0]=sum(temp<=1)
        observed[1]=sum(temp==2)
        
        if strict:
            if results[results.argmax()]==total:
                val_list.append(results.argmax())
            else:
                val_list.append(None)
        else:
            if observed[observed.argmax()]==total:
                val_list.append(results.argmax())
            else:
                val_list.append(None)
    if dropna:
        return [(k,v) for k,v in zip(data_dict.keys(),val_list) if v is not None]
    else:
        return [(k,v) for k,v in zip(data_dict.keys(),val_list) ]
    

In [23]:
# <tho High; tho<=;<=tho1; moderate; >tho2 Low
def process_df(inh,cutoff=1.25):
    inh_cell_fda,_=reset_df(inh)
    flag_list = []
    data_list = []
    #2.0
    tho2 = cutoff
    tho3 = 1.25
    l=np.array(inh_cell_fda['value_num'].tolist())
    for i in range(len(inh_cell_fda)):
        data = inh_cell_fda.iloc[i]
        param,val,unit = data['parameter'],data['value'],data['units']

        eps = 1e-6

        if ('<' in val and l[i] < tho2):
            data_list.append(0)
        elif ('>' in val and l[i] >= tho3):
            data_list.append(2)
        elif ('>' in val or '<' in val):
            data_list.append(None)
        elif l[i]<tho2:
            data_list.append(0)
        elif l[i]>=tho2 and l[i]<tho3:
            data_list.append(1)
        else:
            data_list.append(2)

    inh_cell_fda['origin'] = inh_cell_fda['value']
    inh_cell_fda['value'] = data_list
    inh_cell_fda=inh_cell_fda.dropna(subset=['value'])
    print('process:',len(inh_cell_fda))
    
    return inh_cell_fda

def pre_merge_df(inh_cell_fda,inh_cell_fda1):
    print('origin:',len(inh_cell_fda))
    inh_cell_fda['clean_smiles_pH'] = [ clean_and_standardize(s)[1] for s in inh_cell_fda['structure'] ]
    data_dict = defaultdict(list)
    smiles_dict = defaultdict(list)
    
    data_dict0 = defaultdict(list)
    for i in range(len(inh_cell_fda)):
        data = inh_cell_fda.iloc[i]
        smiles = data['clean_smiles_pH']
        smiles_dict[smiles].append(data['drug'])
        sub = data['substanceMeasured']
        exp = data['experimentalSystem']
        param,val,unit = data['parameter'],data['value'],data['units']
        data_dict0['_'.join([smiles,sub,exp])].append(val)
    
    for k,v in test_v1(data_dict0):
        data_dict[k].append(v)
    
    data_dict1 = defaultdict(list)
    for i in range(len(inh_cell_fda1)):
        data = inh_cell_fda1.iloc[i]
        smiles = data['clean_smiles_pH']
        smiles_dict[smiles].append(data['drug'])
        sub = data['substanceMeasured']
        exp = data['experimentalSystem']
        param,val,unit = data['parameter'],data['value'],data['units']
        data_dict1['_'.join([smiles,sub,exp])].append(val)
        
    for k,v in test_v1(data_dict1):
        data_dict[k].append(v)
        
    val_list = []
    for smi in data_dict.keys():
        temp = np.array(data_dict[smi])
        val_list.append(temp[-1])

    keys = list(data_dict.keys())
    drug = []
    subs = []
    exps = []
    smiles = []
    for key in keys:
        smi,sub,exp = key.split('_')
        subs.append(sub)
        exps.append(exp)
        drug.append(smiles_dict[smi][0])
        smiles.append(smi)
    df_process = pd.DataFrame({'drug':drug,'smiles':smiles,'sub':subs,'exp':exps,'label':val_list})
    print('process:',len(df_process))
    return df_process

def merge_df(inh_cell_fda):
    print('origin:',len(inh_cell_fda))
    data_dict = defaultdict(list)
    smiles_dict = defaultdict(list)
    for i in range(len(inh_cell_fda)):
        data = inh_cell_fda.iloc[i]
        smiles,sub,exp,val = data['smiles'],data['sub'],data['exp'],data['label']
        smiles_dict[smiles].append(data['drug'])
        data_dict['_'.join([smiles,sub])].append(val)

    # merge different cell's data
    val_list = []
    for k,v in test_v1(data_dict,dropna=False,strict=False):
        val_list.append(v)

    keys = list(data_dict.keys())
    drug = []
    subs = []
    smiles = []
    for key in keys:
        smi,sub = key.split('_')
        subs.append(sub)
        drug.append(smiles_dict[smi][0])
        smiles.append(smi)
    df_process = pd.DataFrame({'drug':drug,'smiles':smiles,'sub':subs,'label':val_list}).dropna()

    data_dict = defaultdict(list)
    smiles_dict = defaultdict(list)
    for i in range(len(inh_cell_fda)):
        data = inh_cell_fda.iloc[i]
        smiles,sub,val = data['smiles'],data['sub'],data['label']
        smiles_dict[smiles].append(data['drug'])
        data_dict[smiles].append(val)

    # val_list = []
    # for k,v in chisquare_fun(data_dict,dropna=False):
    #     val_list.append(v)
    
    # merge different sub's data
    val_list = []
    for k,v in test_v1(data_dict,dropna=False,strict=False):
        val_list.append(v)

    smiles = list(data_dict.keys())
    drug = [ smiles_dict[smi][0] for smi in smiles ]
    df_process = pd.DataFrame({'drug':drug,'smiles':smiles,'label':val_list}).dropna()
    print('process:',len(df_process))
    return df_process

def check_df(inh_cell_fda):
    for keys in inh_cell_fda['sub'].unique():
        data=defaultdict(list)
        flag=[]
        smi_dict=defaultdict(str)
        print(keys)
        for i in range(len(inh_cell_fda)):
            exp = inh_cell_fda.iloc[i]['drug']
            smi = inh_cell_fda.iloc[i]['smiles']
            sub = inh_cell_fda.iloc[i]['sub']
            val = inh_cell_fda.iloc[i]['label']

            if ((keys == sub) and (
                'cein' in exp or
                'goxin' in exp or
                'damine' in exp or
                'blastine' in exp or
                'chst' in exp or
                'phine' in exp or
                'icine' in exp or
                'chst' in exp or
                'taxel' in exp or
                'bicin' in exp ) 
            ):
                # print(exp,val)
                flag.append(True)
                data[smi+'_'+sub].append(val)
                smi_dict[smi+'_'+sub]=exp
                # data.append(val)
            else:
                flag.append(False)

        test_list=[]
        for k,v in data.items():
            p_value_max=round(stats.ttest_1samp(v, np.array(v).max())[1],3)
            p_value_min=round(stats.ttest_1samp(v, np.array(v).min())[1],3)
            print(smi_dict[k],k.split('_')[1],len(v),v)
        print()
        
# df_vivo = df_vivo[df_vivo['label']]
def calc_confict(df_process,df_vivo):
    data_dict = defaultdict(list)
    smiles_dict = defaultdict(list)
    for i in range(len(df_process)):
        data = df_process.iloc[i]
        smiles = data['smiles']
        smiles_dict[smiles].append(data['drug'])
        val = data['label']
        data_dict[smiles].append(val)

    for i in range(len(df_vivo)):
        data = df_vivo.iloc[i]
        smiles = data['smiles']
        smiles_dict[smiles].append(data['drug'])
        val = data['label']
        data_dict[smiles].append(val)

    count=defaultdict(list)
    for v in data_dict.values():
        if len(v)==2:
            count[v[1]].append(v[0]!=v[1])
            
    val_list=[]
    for v in data_dict.values():
        val_list.append(v[-1])
        
    smiles = list(data_dict.keys())
    drug = [ smiles_dict[smi][0] for smi in smiles ]
    temp = pd.DataFrame({'drug':drug,'smiles':smiles,'label':val_list}).dropna()

    total=[]
    for k,v in count.items():
        print('label:',k)
        if len(v)>0:
            print(sum(v),len(v),sum(v)/len(v))
        total.append(sum(v)/len(v))
    if len(total)>0:
        print('total')
        print(total,sum(total)/2)
        
    return temp

In [24]:
from copy import deepcopy

In [25]:
# 0.85-1.25; 1.25-
# 0.85-1.125; 1.125-1.25; 1.25-
inh_cell_fda1=process_df(inh,1.25-0.125)
print('limit',count_place(inh_cell_fda1,'value'))
print()
inh_cell_fda=pre_merge_df(inh_cell_fda1,inh_cell_fda1)
print('pre_merge',count_place(inh_cell_fda,'label'))
print()
check_df(inh_cell_fda)

# 1: inh
# 2: potential non-inh
# 3: non_inh
inh_cell_fda=merge_df(inh_cell_fda)
print('merge',count_place(inh_cell_fda,'label'))
inh_cell_fda_test=deepcopy(inh_cell_fda)
inh_cell_fda_test['label']=[ True if s==2 else (False if s==0 else False) for s in inh_cell_fda['label'].tolist()]
inh_cell_fda['label']=[ True if s==2 else (False if s==0 else None) for s in inh_cell_fda['label'].tolist()]
inh_cell_fda_test=inh_cell_fda_test.dropna()
inh_cell_fda=inh_cell_fda.dropna()

print('merge_binary',count_place(inh_cell_fda,'label'))
print('merge_binary_test',count_place(inh_cell_fda_test,'label'))

origin: 200
process: 200
200 {'ratio': 200}
process: 199
limit {2.0: 78, 0.0: 75, 1.0: 46}

origin: 199
process: 63
pre_merge {0: 32, 2: 22, 1: 9}

Digoxin

origin: 63
process: 63
merge {0: 32, 2: 22, 1: 9}
merge_binary {False: 32, True: 22}
merge_binary_test {False: 41, True: 22}


In [26]:
inh_cell_fda.to_csv('inhibitors_vivo.csv',index=False)
inh_cell_fda_test.to_csv('inhibitors_vivo_test.csv',index=False)